In [17]:
from extractors.pdf_extractor import extract_pdf_as_markdown,extract_docx_as_markdown
from chains.audit_chain import analyze_module
from chains.evidence_chain import analyze_evidence
from chains.summary_report_chain import get_summary
from chains.improvement_chain import get_improvement_chain
from langchain_community.chat_models import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
import asyncio
import re
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [23]:
load_dotenv()
# print(os.getenv("OPENAI_API_KEY"))

file_path = "./files/"
controls = pd.read_json("./data/ISO_27001_2022_Controls_List.json")


In [24]:
controls

,Section,Control ID,Control Title,Clause
0,A.5 (Organisational Controls),A.5.1,Policies for information security,"4,5"
1,A.5 (Organisational Controls),A.5.2,Information security roles and responsibilities,"4,5"
2,A.5 (Organisational Controls),A.5.3,Segregation of duties,5
3,A.5 (Organisational Controls),A.5.4,Management responsibilities,"5,6,9,10"
4,A.5 (Organisational Controls),A.5.5,Contact with authorities,
...,...,...,...,...
88,A.8 (Technological Controls),A.8.30,Outsourced development,
89,A.8 (Technological Controls),A.8.31,"Separation of development, test and production...",6
90,A.8 (Technological Controls),A.8.32,Change management,8
91,A.8 (Technological Controls),A.8.33,Test information,


In [ ]:
import numpy as np

n = len(controls)
part_size = n // 5
remainder = n % 5

# Calculate split indices
sizes = [part_size + (1 if i < remainder else 0) for i in range(5)]
indices = np.cumsum([0] + sizes)

dfs = [controls.iloc[indices[i]:indices[i+1]].reset_index(drop=True) for i in range(5)]
df1, df2, df3, df4, df5 = dfs

In [22]:
dfs == [df1, df2, df3, df4, df5]

True

In [12]:
def extract_file_as_markdown(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        return extract_pdf_as_markdown(file_path)
    elif ext == ".docx":
        return extract_docx_as_markdown(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

In [13]:
def read_folder_and_join_markdown(folder_path, file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        file_list (list[str], optional): List of filenames (with or without relative paths) to include.
                                         If None, include all .pdf/.docx files.

    Returns:
        str: Combined markdown string from all valid files.
    """
    all_markdown = []
    normalized_file_list = set(os.path.normpath(f).lower() for f in file_list) if file_list else None
    print(f"Normalized file list: {normalized_file_list}")

    for root, _, files in os.walk(folder_path):
        for filename in files:
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_file_list and normalized_relative_path not in normalized_file_list:
                continue  # Skip files not in the list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)
def read_folder_and_join_markdown_exclude(folder_path, exclude_file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        exclude_file_list (list[str], optional): List of file paths (relative to folder_path) to exclude.

    Returns:
        str: Combined markdown string from all valid, non-excluded files.
    """
    all_markdown = []
    normalized_exclude_list = set(os.path.normpath(f).lower() for f in exclude_file_list) if exclude_file_list else set()

    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.startswith("._"):
                continue  
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_relative_path in normalized_exclude_list:
                continue  # Skip files in the exclusion list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)

In [14]:
with open("output.md", 'r', encoding='utf-8') as f:
    evidence_text = f.read()
with open("policy_text.md", 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx","Initial Implementation Plan - AI-Driven Document Compliance Analysis System (3).pdf"]
# file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx"]
evidence_text = read_folder_and_join_markdown_exclude(file_path, exclude_file_list=file_list)
text = read_folder_and_join_markdown(file_path,file_list=file_list)


✔️ Tesseract detected: tesseract 4.1.1


KeyboardInterrupt: 

In [8]:
len(evidence_text)

68049

In [9]:

# llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
# clauses = [df1,df2,df3,df4,df5]
# audits = []
# results = []
# for clause in clauses:
#     audit_chain = analyze_module(llm)
#     results.append(audit_chain.run(text=text, control_json=clause.to_json(orient='records', indent=2)))



In [15]:
from langchain.callbacks import get_openai_callback
import nest_asyncio
import sys
nest_asyncio.apply()

# text = extract_pdf_as_markdown(file_path)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1,df2,df3,df4,df5]
results = []
total_tokens = 0
total_cost = 0.0

/tmp/ipykernel_2191/2118181688.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))


In [12]:


async def process_clause_async(clause, text, llm):
    def sync_callback_wrapper():
        with get_openai_callback() as cb:
            audit_chain = analyze_module(llm)
            result = audit_chain.run(
                text=text,
                control_json=clause.to_json(orient='records', indent=2)
            )
            return {
                "result": result,
                "tokens": cb.total_tokens,
                "cost": cb.total_cost,
                "controls": len(clause)
            }

    return await asyncio.to_thread(sync_callback_wrapper)

async def run_all_clauses():
    results = []
    total_tokens = 0
    total_cost = 0.0

    tasks = [process_clause_async(clause, text, llm) for clause in clauses]
    clause_results = await asyncio.gather(*tasks)

    for data in clause_results:
        results.append(data["result"])
        total_tokens += data["tokens"]
        total_cost += data["cost"]
        print(f"🔎 Clause Processed: {data['controls']} controls")
        print(f"🧠 Tokens used: {data['tokens']}")
        print(f"💵 Cost: ${data['cost']:.6f}")

    print("\n✅ All clauses processed (async).")
    print(f"🔢 Total tokens used: {total_tokens}")
    print(f"💰 Total cost: ${total_cost:.6f}")

    return results

if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    final_results = await run_all_clauses() if 'google.colab' in sys.modules or 'IPython' in sys.modules else asyncio.run(run_all_clauses())
    


/home/sherry/DocAnalyzer/Document_Analyzer_Streamlit_App/chains/audit_chain.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_2252/2963450692.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = audit_chain.run(


🔎 Clause Processed: 19 controls
🧠 Tokens used: 32141
💵 Cost: $0.016192
🔎 Clause Processed: 19 controls
🧠 Tokens used: 33245
💵 Cost: $0.017912
🔎 Clause Processed: 19 controls
🧠 Tokens used: 32172
💵 Cost: $0.016288
🔎 Clause Processed: 18 controls
🧠 Tokens used: 32594
💵 Cost: $0.017020
🔎 Clause Processed: 18 controls
🧠 Tokens used: 32039
💵 Cost: $0.016112

✅ All clauses processed (async).
🔢 Total tokens used: 162191
💰 Total cost: $0.083524


In [8]:
import json
def load_json_report(results):
    # Step 1: Extract content from wrapper
    if isinstance(results, list):
        if len(results) == 1 and isinstance(results[0], str):
            raw_text = results[0]
        elif all(isinstance(item, dict) for item in results):
            return results
        else:
            raw_text = ''.join(results)
    elif isinstance(results, str):
        raw_text = results
    else:
        raw_text = str(results)
 
    # Step 2: Remove markdown fences
    raw_text = raw_text.strip()
    if raw_text.startswith("```json"):
        raw_text = raw_text.removeprefix("```json").strip()
    if raw_text.endswith("```"):
        raw_text = raw_text.removesuffix("```").strip()
 
    # Step 3: Use regex to extract all JSON arrays
    try:
        # Extract all top-level lists using regex
        arrays = re.findall(r'\[\s*{.*?}\s*\]', raw_text, re.DOTALL)
        combined = []
        for arr in arrays:
            data = json.loads(arr)
            if isinstance(data, list):
                combined.extend(data)
            else:
                combined.append(data)
        return combined
 
    except json.JSONDecodeError as e:
        print("❌ JSON parsing failed:", e)
        print("--- JSON Preview ---")
        print(raw_text[:500])
        raise

In [9]:
final_results = pd.read_json("policy.json")

In [ ]:
# parsed_final_results = load_json_report(final_results)
# final_result_df= pd.DataFrame(parsed_final_results)

In [16]:



# Step 2: Serialize to JSON string

filtered = json.dumps(final_result_df[["Control Id", "Clause", "Control Title", "Policy"]].to_dict(orient='records'), indent=2)

In [ ]:
evidence_chain = analyze_evidence(llm)
result = evidence_chain.run(
    text=evidence_text,
    control_json=filtered
)

In [18]:
result

'[\n  {\n    "Control Id": "A.5.1",\n    "Policy Implementation Evidence": "Managment Review Team Meeting E-WISE 2025 [Restricted].docx.pdf",\n    "Exact Evidence Extract": "Information Security Policy is referenced as an internal document relevant to the management review meeting (Page 16). The policy is shared with all employees and information security goals are communicated and added to training (Page 14).",\n    "Evidence Recommendation": "Include explicit evidence of the policy approval by the director and documented yearly review by ISSC and ISWG in the evidence.",\n    "Evidence": "🟡"\n  },\n  {\n    "Control Id": "A.5.2",\n    "Policy Implementation Evidence": "2.4.7 Template checklist indienst.PNG.pdf and 2.4.8 Template checklist uitdienst.PNG.pdf",\n    "Exact Evidence Extract": "Onboarding and offboarding checklists define roles and responsibilities such as issuing keys, setting up accounts, and returning assets (Pages 1). Roles like Operational Director and System Administ

In [19]:
import json
results=result
with open("evidence_report.json", "w") as f:
    f.write(json.dumps(results, indent=2, ensure_ascii=False))

In [20]:
results

'[\n  {\n    "Control Id": "A.5.1",\n    "Policy Implementation Evidence": "Managment Review Team Meeting E-WISE 2025 [Restricted].docx.pdf",\n    "Exact Evidence Extract": "Information Security Policy is referenced as an internal document relevant to the management review meeting (Page 16). The policy is shared with all employees and information security goals are communicated and added to training (Page 14).",\n    "Evidence Recommendation": "Include explicit evidence of the policy approval by the director and documented yearly review by ISSC and ISWG in the evidence.",\n    "Evidence": "🟡"\n  },\n  {\n    "Control Id": "A.5.2",\n    "Policy Implementation Evidence": "2.4.7 Template checklist indienst.PNG.pdf and 2.4.8 Template checklist uitdienst.PNG.pdf",\n    "Exact Evidence Extract": "Onboarding and offboarding checklists define roles and responsibilities such as issuing keys, setting up accounts, and returning assets (Pages 1). Roles like Operational Director and System Administ

In [21]:

import json# Step 3: Convert to DataFrame
parsed_results = load_json_report(results)

resutls_df = pd.DataFrame(parsed_results)


In [22]:
resutls_df

,Control Id,Policy Implementation Evidence,Exact Evidence Extract,Evidence Recommendation,Evidence
0,A.5.1,Managment Review Team Meeting E-WISE 2025 [Res...,Information Security Policy is referenced as a...,Include explicit evidence of the policy approv...,🟡
1,A.5.2,2.4.7 Template checklist indienst.PNG.pdf and ...,Onboarding and offboarding checklists define r...,Provide a formal organizational chart or role/...,🟡
2,A.5.3,Managment Review Team Meeting E-WISE 2025 [Res...,Segregation of duties is tied to Zoho rights a...,Include documented compensating controls where...,🟡
3,A.5.4,Managment Review Team Meeting E-WISE 2025 [Res...,Management review meetings cover security obje...,None,✅
4,A.5.5,Managment Review Team Meeting E-WISE 2025 [Res...,Contact details and responsibilities for autho...,Provide explicit documented procedures or cont...,🟡
...,...,...,...,...,...
88,A.8.30,A.5.8/4.4.1.6.1 System development life cycle ...,Outsourced development contracts include licen...,None,✅
89,A.8.31,A.5.8/4.4.1.6.1 System development life cycle ...,"Development, test, and production environments...",None,✅
90,A.8.32,Managment Review Team Meeting E-WISE 2025 [Res...,Changes to systems and software controlled via...,None,✅
91,A.8.33,A.5.8/4.4.1.6.1 System development life cycle ...,"Test environments use limited, anonymised or p...",None,✅


In [23]:
final_result_df

,Clause,Section,Control Id,Control Title,Compliance,Policy,Reference,Gaps Identified,Recommended Action
0,"4,5",A.5 (Organisational Controls),A.5.1,Policies for information security,✅,The Information Security Policy document defin...,Information Security Policy [Internal].docx - ...,,
1,"4,5",A.5 (Organisational Controls),A.5.2,Information security roles and responsibilities,✅,Roles and responsibilities are clearly defined...,Information Security Policy [Internal].docx - ...,,
2,5,A.5 (Organisational Controls),A.5.3,Segregation of duties,✅,Segregation of duties is required when assigni...,Information Security Policy [Internal].docx - ...,,
3,"5,6,9,10",A.5 (Organisational Controls),A.5.4,Management responsibilities,✅,Management is responsible for setting security...,Information Security Policy [Internal].docx - ...,,
4,,A.5 (Organisational Controls),A.5.5,Contact with authorities,✅,Procedures exist for contacting local governme...,Information Security Policy [Internal].docx - ...,,
...,...,...,...,...,...,...,...,...,...
88,,A.8 (Technological Controls),A.8.30,Outsourced development,✅,Outsourced development contracts include licen...,Section 8.1.2 - Licensing and IP rights agreed...,,
89,6,A.8 (Technological Controls),A.8.31,"Separation of development, test and production...",✅,"Development, test, and production environments...",Section 8.1.14-15 - Development and test syste...,,
90,8,A.8 (Technological Controls),A.8.32,Change management,✅,Changes to systems and software are controlled...,Section 6.2 Operational change control - docum...,,
91,,A.8 (Technological Controls),A.8.33,Test information,✅,"Test environments use limited, anonymised or p...",Section 8.1.10 and 8.1.21 - Test environments ...,,


In [24]:
final_result_df.to_json("policy.json")

In [25]:
parsed_final_results

[{'Clause': '4,5',
  'Section': 'A.5 (Organisational Controls)',
  'Control Id': 'A.5.1',
  'Control Title': 'Policies for information security',
  'Compliance': '✅',
  'Policy': 'The Information Security Policy document defines baseline control measures, scope, enforcement, and responsibilities. It is approved by the director and reviewed yearly by ISSC and ISWG. It covers confidentiality, compliance expectations, and management oversight.',
  'Reference': 'Information Security Policy [Internal].docx - Sections: Confidentiality Statement, Chapter 1 General, Section 1.1 Scope, Section 1.4 Enforcements, Chapter 10 Evaluation and Improvement (Section 10.1 Review of information security).',
  'Gaps Identified': '',
  'Recommended Action': ''},
 {'Clause': '4,5',
  'Section': 'A.5 (Organisational Controls)',
  'Control Id': 'A.5.2',
  'Control Title': 'Information security roles and responsibilities',
  'Compliance': '✅',
  'Policy': 'Roles and responsibilities are clearly defined for CEO,

In [26]:
merged_df = pd.merge(final_result_df, resutls_df, on="Control Id", how="outer")


In [27]:
merged_df['Evidence'] = merged_df['Evidence'].replace('', pd.NA).fillna('❌')

In [28]:
merged_df

,Clause,Section,Control Id,Control Title,Compliance,Policy,Reference,Gaps Identified,Recommended Action,Policy Implementation Evidence,Exact Evidence Extract,Evidence Recommendation,Evidence
0,"4,5",A.5 (Organisational Controls),A.5.1,Policies for information security,✅,The Information Security Policy document defin...,Information Security Policy [Internal].docx - ...,,,Managment Review Team Meeting E-WISE 2025 [Res...,Information Security Policy is referenced as a...,Include explicit evidence of the policy approv...,🟡
1,6,A.5 (Organisational Controls),A.5.10,Acceptable use of information and other associ...,✅,Acceptable use policies are defined in the Inf...,Information Security Policy [Internal].docx - ...,,,Managment Review Team Meeting E-WISE 2025 [Res...,Acceptable use policies are enforced; misuse r...,Include direct reference or excerpt of the Acc...,🟡
2,,A.5 (Organisational Controls),A.5.11,Return of assets,✅,Procedures exist for return of company assets ...,Information Security Policy [Internal].docx - ...,,,2.4.8 Template checklist uitdienst.PNG.pdf,Checklist for offboarding includes return of k...,None,✅
3,6,A.5 (Organisational Controls),A.5.12,Classification of information,✅,Information classification schemes are establi...,Information Security Policy [Internal].docx - ...,,,No explicit evidence found in provided documents.,No direct evidence of information classificati...,Provide documented classification scheme and e...,❌
4,,A.5 (Organisational Controls),A.5.13,Labelling of information,✅,Information must be labelled and handled accor...,Information Security Policy [Internal].docx - ...,,,No explicit evidence found in provided documents.,No direct evidence of labelling of information...,Provide documented labelling guidelines and ev...,❌
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,,A.8 (Technological Controls),A.8.5,Secure authentication,✅,"Section 4.1 mandates password confidentiality,...",Information Security Policy [Internal].docx - ...,,,Kopie van BS02 E-Wise Wachtwoordbeleid [Intern...,"Password confidentiality, complexity, change r...",None,✅
89,8,A.8 (Technological Controls),A.8.6,Capacity management,✅,Section 3.3.1 and 3.3.2 of Business Continuity...,Business Continuity Plan E-WISE [Restricted].d...,,,Managment Review Team Meeting E-WISE 2025 [Res...,Capacity management included in Business Conti...,None,✅
90,8,A.8 (Technological Controls),A.8.7,Protection against malware,✅,Section 6.6 mandates installation and maintena...,Information Security Policy [Internal].docx - ...,,,Managment Review Team Meeting E-WISE 2025 [Res...,Antivirus software installed and maintained; u...,None,✅
91,8,A.8 (Technological Controls),A.8.8,Management of technical vulnerabilities,✅,Section 6.2 defines a vulnerability management...,Information Security Policy [Internal].docx - ...,,,Managment Review Team Meeting E-WISE 2025 [Res...,Vulnerability management process defined inclu...,None,✅


In [29]:
merged_df.to_excel("result.xlsx")

In [30]:
with open("output.md", "w", encoding="utf-8") as f:
    f.write(evidence_text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'


In [31]:
with open("policy_text.md", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'


In [12]:
%pip install python-docx pypandoc beautifulsoup4 beautifulsoup4 striprtf pdfkit weasyprint


  Using cached cffi-1.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached tinycss2-1.4.0-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 537.7 kB/s eta 0:00:00a 0:00:01
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 1.6 MB/s eta 0:00:0000:0100:01
Using cached cffi-1.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (479 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:0000:0100:01
Using cached tinycss2-1.4.0-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 6.0 MB/s eta 0:00:0000:0100:01m
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))


/tmp/ipykernel_18132/1769623979.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))


In [3]:
import pandas as pd
from docx import Document

df = pd.read_excel("result.xlsx")
doc = Document("ISO 27001 - Valecta - Template - Report.docx")
docx_text = "\n".join([para.text for para in doc.paragraphs if para.text.strip() != ""])



In [9]:
summary_chain = get_summary(llm)
result = summary_chain.run(
    report=docx_text,
    assessment=df.to_json(orient="records", indent=2)
)

In [13]:
result

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8" />\n  <meta name="viewport" content="width=device-width, initial-scale=1" />\n  <title>ISO 27001:2022 Gap Analyse - Vlectra</title>\n  <style>\n    @import url(\'https://fonts.googleapis.com/css2?family=Inter:wght@400;600&display=swap\');\n    body {\n      font-family: \'Inter\', sans-serif;\n      background: #f9fafb;\n      margin: 0;\n      padding: 2rem;\n      color: #1f2937;\n    }\n    h1, h2, h3, h4 {\n      color: #111827;\n      margin-bottom: 0.5rem;\n    }\n    h1 {\n      font-size: 2.25rem;\n      border-bottom: 3px solid #3b82f6;\n      padding-bottom: 0.25rem;\n      margin-bottom: 1.5rem;\n    }\n    h2 {\n      font-size: 1.5rem;\n      margin-top: 2rem;\n      border-bottom: 2px solid #93c5fd;\n      padding-bottom: 0.25rem;\n    }\n    h3 {\n      font-size: 1.25rem;\n      margin-top: 1.5rem;\n      color: #2563eb;\n    }\n    p {\n      line-height: 1.6;\n      margin-top: 0.5rem;\n      margin-b

In [14]:
from weasyprint import HTML
from bs4 import BeautifulSoup
import os

# Assuming `result` contains your HTML string
soup = BeautifulSoup(result, 'html.parser')

# Get absolute file path and convert to file URL
abs_path = os.path.abspath("Vlectra_logo.jpg")
file_url = f"file://{abs_path}"

# Insert image with absolute file path
img_tag = soup.new_tag(
    "img",
    src=file_url,  # or base64 version
    alt="Logo",
    style="width: 150px; height: auto;"  # Adjust as needed
)
soup.body.insert(0, img_tag)

# Generate PDF
HTML(string=str(soup)).write_pdf("output.pdf")


In [15]:
from docx.shared import Inches
import pypandoc
from striprtf.striprtf import rtf_to_text
from docx import Document
from bs4 import BeautifulSoup

In [13]:
import binascii

def image_to_hex_rtf(image_path):
    with open(image_path, 'rb') as f:
        hex_data = binascii.hexlify(f.read()).upper()
        hex_string = hex_data.decode('ascii')

        # RTF expects line breaks every ~64 characters
        hex_lines = [hex_string[i:i+64] for i in range(0, len(hex_string), 64)]
        return '\n'.join(hex_lines)

def create_rtf_with_image(image_path, original_rtf_text):
    image_hex = image_to_hex_rtf(image_path)

    # Minimal RTF image header for PNG or JPEG (set image type and optional size info)
    pict_header = (
        r"{\pict\pngblip"  # Use \jpegblip for JPEG
        r"\picw1000\pich1000"  # Logical width/height in twips (optional)
        "\n" + image_hex + "\n}"
    )

    # Build the final RTF: keep original header and add image at the top of the body
    if original_rtf_text.startswith("{\\rtf"):
        # Insert image after RTF header and before any body content
        parts = original_rtf_text.split('\n', 1)
        rtf_with_image = parts[0] + '\n' + pict_header + '\n' + (parts[1] if len(parts) > 1 else '')
        return rtf_with_image
    else:
        raise ValueError("Invalid RTF format")

# Example usage:
image_path = "Vlectra_logo.jpg"


rtf_with_image = create_rtf_with_image(image_path, result)

# Save new RTF
with open("temp_input.rtf", "w", encoding="utf-8") as f:
    f.write(rtf_with_image)



In [10]:
!sudo apt install wkhtmltopdf

[sudo] password for sherry: 
sudo: a password is required
^C
